# Laboratorio 4 - Parte 1

### Redes Neuronales Artificiales: MLP

### 2018-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: John Alexander Galeano Ospina
#### Segundo Integrante: Sidney Paola Aguirre Castro

In [55]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt
from __future__ import division

#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

## Indicaciones

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de regresión. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

Para este ejercicio usaremos la base de datos sobre calidad del aire, que ha sido usada en laboratorios previos, pero en este caso trataremos de predecir dos variables en lugar de una, es decir, abordaremos un problema de múltiples salidas.

In [56]:
#cargamos la bd que está en un archivo .data y ahora la podemos manejar de forma matricial
db = np.loadtxt('DB/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter

#Esta es la base de datos AirQuality del UCI Machine Learning Repository. En la siguiente URL se encuentra toda
#la descripción de la base de datos y la contextualización del problema.
#https://archive.ics.uci.edu/ml/datasets/Air+Quality#

X = db[:,0:11]
Y = db[:,11:13]

In [59]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape 

## Ejercicio 1

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de regresión propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [33]:
from numpy import random
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
#Validamos el modelo
Folds = 4
random.seed(19680801)
ErrorY1 = np.zeros(Folds)
ErrorY2 = np.zeros(Folds)
ErrorT = np.zeros(Folds)
ss = ShuffleSplit(n_splits=Folds, test_size=0.3)
j = 0
for train, test in ss.split(X):
    Xtrain = X[train,:]
    Ytrain = Y[train,:]
    Xtest = X[test,:]
    Ytest = Y[test,:]
    
    #Normalizamos los datos
    media = np.mean(Xtrain,axis=0)
    desvia = np.std(Xtrain,axis=0)
    Xtrain = preprocessing.scale(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    epochs = 500
    neurons = 36
    mlp = MLPRegressor(hidden_layer_sizes=(neurons,neurons),activation='tanh', max_iter=epochs)
    mlp.fit(Xtrain,Ytrain)
    fnO = mlp.out_activation_ #imprime nombre de función de activación en la capa de salida
    
    #Use para el modelo para hacer predicciones sobre el conjunto Xtest
    Yest = mlp.predict(Xtest)
    
    #Mida el error MAPE para cada una de las dos salidas
    ErrorY1[j] = MAPE(Yest[:,0], Ytest[:,0])
    ErrorY2[j] = MAPE(Yest[:,1], Ytest[:,1])
    ErrorT[j] = (ErrorY1[j] + ErrorY2[j])/2
    j += 1

print(fnO)
#print(Yest.shape)  
print('MAPE salida 1 = ' + str(np.mean(ErrorY1)) + '+-' + str(np.std(ErrorY1)))
print('MAPE salida 2 = ' + str(np.mean(ErrorY2)) + '+-' + str(np.std(ErrorY2)))
print('MAPE total = '    + str(np.mean(ErrorT)) + '+-' + str(np.std(ErrorT)))

identity
MAPE salida 1 = 0.06600835243363894+-0.0016428393530221281
MAPE salida 2 = 1.0220498948011731+-0.07211487660609449
MAPE total = 0.5440291236174061+-0.03573485899945333


In [50]:
Yest[:,0].reshape(2808,1).shape
Y.shape

(array([-2.000e+02,  1.847e-01,  1.862e-01, ...,  8.710e+01,  8.720e+01,
         8.870e+01]), array([732,   1,   1, ...,   1,   1,   1], dtype=int64))

## Ejercicio 2

Una vez completado el código anterior. Realice los experimentos necesarios para completar la tabla siguiente:

In [20]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["MAPE salida 1"] = ""
df_types["IC MAPE salida 1"] = ""
df_types["MAPE salida 2"] = ""
df_types["IC MAPE salida 2"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [32]:
qgrid_widget.get_changed_df()

MAPE salida 1 IC MAPE salida 1  \
N. de capas ocultas Neuronas por capa                                  
1                   20                       0.0862           0.0019   
                    24                       0.0820           0.0015   
                    28                       0.0772           0.0014   
                    32                       0.0769           0.0015   
                    36                       0.0756           0.0007   
2                   20                       0.0714           0.0031   
                    24                       0.0662           0.0005   
                    28                       0.0678           0.0009   
                    32                       0.0672           0.0029   
                    36                       0.0660           0.0016   

                                      MAPE salida 2 IC MAPE salida 2  
N. de capas ocultas Neuronas por capa                                 
1                   20                       2.3985           0.1585  
                    24                       2.0704           0.1187  
                    28                       1.7329           0.0694  
                    32                       1.5158           0.0524  
                    36                       1.4609           0.0753  
2                   20                       1.2207           0.0997  
                    24                       1.1099           0.0867  
                    28                       1.0028           0.0338  
                    32                       1.0943           0.1408  
                    36                       1.0220           0.0721

<b>Responda</b>:

2.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 

R: En la capa de salida, según la librería de sklearn usada, el modelo usa la función de activación identidad, pero como esto quiere decir que a cada valor de x al que se le aplica la función, retorna el mismo x, concluímos que en teoría el modelo no usa función de activación en la capa de salida, ya que es una regresión, y la salida es un valor continuo.

## Ejercicio 3.

A continuación se leen los datos de un problema de clasificación. El problema corresponde a la clasifiación de dígitos escritos a mano. Usaremos únicamente 4 de las 10 clases disponibles. Los datos fueron preprocesados para reducir el número de características. La técnica usada será analizada más adelante en el curso.

In [64]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de clasificación. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de clasificación propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [65]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    
    #Normalizamos los datos
    media = np.mean(Xtrain)
    desvia = np.std(Xtrain)
    Xtrain = preprocessing.scale(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    epochs = 500
    neurons = 36
    mlp = MLPClassifier(hidden_layer_sizes=(neurons,neurons ), activation='tanh', max_iter=epochs)
    mlp.fit(Xtrain,Ytrain)
    fnO = mlp.out_activation_ #imprime nombre de función de activación en la capa de salida
    nO = mlp.n_outputs_ #imprime número de salidas en la capa de salida
    
    #Validación con las muestras de entrenamiento
    Ytrain_pred =  mlp.predict(Xtrain)
     
    #Validación con las muestras de test    
    Yest = mlp.predict(Xtest)
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
    EficienciaVal[j] = np.mean(Yest == Ytest)
    j += 1
print('Función en la capa de salida: '+ str(fnO))  
print('Cantidad de neuronas en la capa de salida: '+ str(nO))
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

Función en la capa de salida: softmax
Cantidad de neuronas en la capa de salida: 4
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9610542147143794+-0.01970756737185292


In [68]:
ncc = np.unique(Y)
nc = len(ncc)
print("hay " + str(nc) + " clases")

hay 4 clases


## Ejercicio 4

Una vez completado el código realice los experimentos necesarios para llenar la siguiente tabla:

In [18]:
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [22]:
qgrid_widget.get_changed_df()

Eficiencia en validacion  \
N. de capas ocultas Neuronas por capa                            
1                   20                                  0.9637   
                    24                                  0.9623   
                    28                                  0.9637   
                    32                                  0.9596   
                    36                                  0.9553   
2                   20                                  0.9693   
                    24                                  0.9526   
                    28                                  0.9512   
                    32                                  0.9568   
                    36                                  0.9610   

                                      Intervalo de confianza  
N. de capas ocultas Neuronas por capa                         
1                   20                                0.0267  
                    24                                0.0369  
                    28                                0.0246  
                    32                                0.0293  
                    36                                0.0339  
2                   20                                0.0225  
                    24                                0.0374  
                    28                                0.0337  
                    32                                0.0349  
                    36                                0.0197

<b>Responda</b>:

4.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 
R: softmax

4.2 ¿Cuántas neuronas en la capa de salida tiene el modelo?¿Por qué debe tener ese número?
R: El modelo tiene 4 neuronas en la capa de salida. Debe tener ese número porque el modelo utiliza una función Softmax en la capa de salida, lo que hace que se active la salida que corresponde a la clase a la que pertenece la muestra. Como el problema tiene 4 clases en las que se va a clasificar cada muestra, entonces, dependiendo de cuál clase es representada por la salida que se activa, significa que la muestra pertenece a dicha clase. 